**9678조**

조원: 이헌수(20163147), 함석민(20163174), 오예린(20171650)

**목적:** 주어진 파일이 악성 파일인지 정상 파일인지 판별하는 이진 분류 모델

**내용**
1. 주어진 json파일들의 특징 추출
  - 주어진 Peminer, Ember 데이터를 일정한 길이의 벡터 형태로 추출
2. 학습
 - 주어진 검증데이터로 학습 진행
3. 결과
 - json 파일들에 대한 악성 여부 판단 후 predict.csv파일 생성

**필요한 라이브러리 IMPORT**

In [ ]:
import os
import glob
import json
import pprint

import numpy as np

from lightgbm import LGBMClassifier

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE

 **필요한 함수들 정의**

In [ ]:
SEED = 41

def read_label_csv(path):
    label_table = dict()
    with open(path, "r", encoding='cp949') as f:
        for line in f.readlines()[1:]:
            fname, label = line.strip().split(",")
            label_table[fname] = int(label)
    return label_table

def read_json(path):
    with open(path, "r") as f:
        return json.load(f)

def load_model(**kwargs):
    if kwargs["model"] == "rf":
        return RandomForestClassifier(random_state=kwargs["random_state"], n_jobs=4)
    elif kwargs["model"] == "dt":
        return DecisionTreeClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "lgb":
        return LGBMClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "svm":
        return SVC(random_state=kwargs["random_state"])
    elif kwargs["model"] == "lr":
        return LogisticRegression(random_state=kwargs["random_state"], n_jobs=-1)
    elif kwargs["model"] == "knn":
        return KNeighborsClassifier(n_jobs=-1)
    elif kwargs["model"] == "adaboost":
        return AdaBoostClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "mlp":
        return MLPClassifier(random_state=kwargs["random_state"])
    else:
        print("Unsupported Algorithm")
        return None
    

def train(X_train, y_train, model):
    '''
        머신러닝 모델을 선택하여 학습을 진행하는 함수
	
        :param X_train: 학습할 2차원 리스트 특징벡터
        :param y_train: 학습할 1차원 리스트 레이블 벡터
        :param model: 문자열, 선택할 머신러닝 알고리즘
        :return: 학습된 머신러닝 모델 객체
    '''
    clf = load_model(model=model, random_state=SEED)
    clf.fit(X_train, y_train)
    return clf


def evaluate(X_test, y_test, model):
    '''
        학습된 머신러닝 모델로 검증 데이터를 검증하는 함수
	
        :param X_test: 검증할 2차원 리스트 특징 벡터
        :param y_test: 검증할 1차원 리스트 레이블 벡터
        :param model: 학습된 머신러닝 모델 객체
    '''
    predict = model.predict(X_test)
    print("정확도", model.score(X_test, y_test))


## 레이블 테이블 로드

In [ ]:
label_table = read_label_csv("학습데이터_정답.csv")
label_table_test= read_label_csv("검증데이터_정답.csv")

## 특징 벡터 생성 예시
- PEMINER 정보는 모두 수치형 데이터이므로 특별히 가공을 하지 않고 사용 가능
- EMBER, PESTUDIO 정보는 가공해서 사용해야 할 특징들이 있음 (e.g. imports, exports 등의 문자열 정보를 가지는 데이터)
- 수치형 데이터가 아닌 데이터(범주형 데이터)를 어떻게 가공할 지가 관건 >> 인코딩 (e.g. 원핫인코딩, 레이블인코딩 등)

In [ ]:
ember_path = "EMBER/학습데이터/000c4ae5e00a1d4de991a9decf9ecbac59ed5582f5972f05b48bc1a1fe57338a.json"
peminer_path = "PEMINER/학습데이터/000c4ae5e00a1d4de991a9decf9ecbac59ed5582f5972f05b48bc1a1fe57338a.json"

ember_result = read_json(ember_path)
peminer_result = read_json(peminer_path)

**특징 분석 및 벡터 생성**

In [ ]:
class PeminerParser:
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []

#    def get_custom_info(self):
#      custom = self.report
#
#      vector = [
#        custom["OptionalHeader.FileAlignment"],
#        custom["OptionalHeader.DllCharacteristics"],
#        custom["Section.text.VirtualSize"],
#       custom["OptionalHeader.SizeOfInitializedData"],
#       custom['OptionalHeader.AddressOfEntryPoint'],
#       custom['OptionalHeader.SectionAlignment'],
#       custom['OptionalHeader.CheckSum']
#     ]

#     return vector
#여러 특징들을 사용하여 벡터를 구성하려 했으니 기본피팅보다 정확도가 낮게 나와 기본 피팅을 하였음

      


    def process_report(self):
        
        #self.vector = self.get_custom_info()
        self.vector = [value for _, value in sorted(self.report.items(), key=lambda x: x[0])]
        return self.vector
    

class EmberParser:
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def get_histogram_info(self):
        histogram = np.array(self.report["histogram"])
        total = histogram.sum()
        vector = histogram / total
        return vector.tolist()
    
    def get_string_info(self):
        strings = self.report["strings"]

        hist_divisor = float(strings['printables']) if strings['printables'] > 0 else 1.0
        vector = [
            strings['numstrings'], 
            strings['avlength'], 
            strings['printables'],
            strings['entropy'], 
            strings['paths'], 
            strings['urls'],
            strings['registry'], 
            strings['MZ']
        ]
        vector += (np.asarray(strings['printabledist']) / hist_divisor).tolist()
        return vector
    
    def get_general_file_info(self):
        general = self.report["general"]
        vector = [
            general['size'], general['vsize'], general['has_debug'], general['exports'], general['imports'],
            general['has_relocations'], general['has_resources'], general['has_signature'], general['has_tls'],
            general['symbols']
        ]
        return vector

    def get_header_info(self):
      header = self.report["header"]["coff"]
      self.vector = [
                header["timestamp"], len(header["machine"]), len(header["characteristics"])]
      
      return self.vector

    def get_imports_info(self):

      imp = self.report["imports"]
      imp_key_list = len(imp.keys())

      vector = [imp_key_list]
      return vector

    def process_report(self):
        vector = []
        vector += self.get_general_file_info()
        vector += self.get_histogram_info()
        vector += self.get_string_info()
        vector += self.get_header_info()
        return vector
    
class PestudioParser:
    '''
        사용할 특징을 선택하여 벡터화 할 것을 권장
    '''
    
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def process_report(self):
        pass

## 학습데이터 구성
- 특징 벡터 구성은 2차원이 되어야함 e.g.  [vector_1, vector_2, ..., vector_n]

- 각 벡터는 1차원 리스트, 벡터 크기는 모두 같아야함

In [ ]:
# 데이터의 특징 벡터 모음(2차원 리스트) : X
# 데이터의 레이블 모음(1차원 리스트) : y

X, y = [], []

train_list = os.listdir('PEMINER/학습데이터')
for fname in train_list:
    feature_vector = []
    fname = fname[:-5]
    label = label_table[fname]
    for data in ["PEMINER/학습데이터", "EMBER/학습데이터"]:
        path = f"{data}/{fname}.json"
        
        if data == "PEMINER/학습데이터":
            feature_vector += PeminerParser(path).process_report()
        else:
            feature_vector += EmberParser(path).process_report()
        
    X.append(feature_vector)
    y.append(label)

np.asarray(X).shape, np.asarray(y).shape


((20000, 561), (20000,))

##검증데이터 구성

In [ ]:
# 데이터의 특징 벡터 모음(2차원 리스트) : X2
# 데이터의 레이블 모음(1차원 리스트) : y2

X2, y2 = [], []

test_list = os.listdir('PEMINER/검증데이터')
for fname in test_list:
    feature_vector = []
    fname = fname[:-5]
    label = label_table_test[fname]
    for data in ["PEMINER/검증데이터", "EMBER/검증데이터"]:
        path = f"{data}/{fname}.json"
        
        if data == "PEMINER/검증데이터":
            feature_vector += PeminerParser(path).process_report()
        else:
            feature_vector += EmberParser(path).process_report()
        
    X2.append(feature_vector)
    y2.append(label)

np.asarray(X2).shape, np.asarray(y2).shape


((10000, 561), (10000,))

In [ ]:
# 데이터의 특징 벡터 모음(2차원 리스트) : X2
# 데이터의 레이블 모음(1차원 리스트) : y2

X_test, y_test = [], []

test_list = os.listdir('PEMINER/테스트데이터')
for fname in test_list:
    feature_vector = []
    fname = fname[:-5]
    for data in ["PEMINER/테스트데이터", "EMBER/테스트데이터"]:
        path = f"{data}/{fname}.json"
        
        if data == "PEMINER/테스트데이터":
            feature_vector += PeminerParser(path).process_report()
        else:
            feature_vector += EmberParser(path).process_report()
        
    X_test.append(feature_vector)

np.asarray(X_test).shape


(10000, 561)

## 학습 및 검증

In [ ]:
# 학습
models = []
for model in ["rf", "lgb"]:
    clf = train(X, y, model)
    models.append(clf)

# 검증
# 실제 검증 시에는 제공한 검증데이터를 검증에 사용해야 함

for model in models:
    evaluate(X2, y2, model)

정확도 0.9434
정확도 0.9529


## 앙상블 예제

In [ ]:
def ensemble_result(X, y, models):
    '''
        학습된 모델들의 결과를 앙상블하는 함수
	
        :param X: 검증할 2차원 리스트 특징 벡터
        :param y: 검증할 1차원 리스트 레이블 벡터
        :param models: 1개 이상의 학습된 머신러닝 모델 객체를 가지는 1차원 리스트
    '''
    
    # Soft Voting
    # https://devkor.tistory.com/entry/Soft-Voting-%EA%B3%BC-Hard-Voting
    predicts = []
    for i in range(len(X)):
        probs = []
        for model in models:
            prob = model.predict_proba(X)[i][1]
            probs.append(prob)
        predict = 1 if np.mean(probs) >= 0.5 else 0
        predicts.append(predict)
    print(predicts)
    print("정확도", accuracy_score(y, predicts))

In [ ]:
ensemble_result(X, y, models)

**predict.csv 파일 생성**

In [ ]:
def predict(X_test,models):
  for model in models:
    predict = model.predict(X_test)
  return predict 

# 테스트 데이터에 대한 예측 값
predict = predict(X_test,models)

import csv


f = open("/content/drive/My Drive/predict.csv",'w')
answer = csv.writer(f)
answer.writerow(['file','predict'])
for i in range(10000):
  answer.writerow([test_list[i][:-5],predict[i]])
f.close()